In [1]:
import csv
import json
from wikiparse import indexer, syntax_parser as sp
import time
import os
import sqlite3

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:/enwiki-20190101-pages-articles-multistream.xml/_actual'

In [2]:
dumps = indexer.load_dumps(build_index=False, scratch_folder='py3')
english = dumps['en']

opening F:/enwiki-20190101-pages-articles-multistream.xml/scratch\py3\index.db
current mapping 19.1 m pages

__init__ complete


In [3]:
c = english.cursor

Before we create the database let's get a complete list of the entries we're going to want. That is, let's look at all the coordinate strings we've extracted from each page and extract the list of keywords from there. 

In [139]:
result = c.execute('''SELECT page_num,coords FROM indices WHERE coords != ""
''').fetchall()
coordStrings = {item[0]:item[1] for item in result}
list(coordStrings.items())[:10]

[(65,
  'Coord|32.7|-86.7|type:adm1st_region:US_dim:1000000_source:USGS|display=title'),
 (66,
  'Coord|32.7|-86.7|type:adm1st_region:US_dim:1000000_source:USGS|display=title'),
 (85, 'Coord|36|42|N|3|13|E|type:city'),
 (86, 'Coord|36|42|N|3|13|E|type:city'),
 (113,
  'Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'),
 (114,
  'Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'),
 (138,
  'Coord|13|19|N|169|9|W|type:event|name=Apollo 11 splashdown||Coord|10|36|N|172|24|E|display=inline||Coord|13|19|N|169|9|W|display=inline'),
 (139,
  'Coord|13|19|N|169|9|W|type:event|name=Apollo 11 splashdown||Coord|10|36|N|172|24|E|display=inline||Coord|13|19|N|169|9|W|display=inline'),
 (140,
  'Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing||Coord|30|12|N|74|7|W|name=Apollo 8 S-IC impact||Coord|31|50|N|37|17|W|name=Apollo 8 S-II impact||Coord|8|8|N|165|1|W

In [12]:
len(coordStrings), type(coordStrings)

(136447, list)

In [141]:
for page_num in coordStrings:
    if '||' in coordStrings[page_num]:
        coordStrings[page_num] = coordStrings[page_num].split('||')[0]

In [142]:
list(coordStrings.items())[:10]

[(65,
  'Coord|32.7|-86.7|type:adm1st_region:US_dim:1000000_source:USGS|display=title'),
 (66,
  'Coord|32.7|-86.7|type:adm1st_region:US_dim:1000000_source:USGS|display=title'),
 (85, 'Coord|36|42|N|3|13|E|type:city'),
 (86, 'Coord|36|42|N|3|13|E|type:city'),
 (113,
  'Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'),
 (114,
  'Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'),
 (138, 'Coord|13|19|N|169|9|W|type:event|name=Apollo 11 splashdown'),
 (139, 'Coord|13|19|N|169|9|W|type:event|name=Apollo 11 splashdown'),
 (140, 'Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing'),
 (160, 'Coord|12|30|40|N|69|58|27|W|type:isle|display=title')]

In [143]:
def getKeywords(coordString, verbose=False):
    if verbose:
        print(coordString)
    keywords = {}
    rest = []
    items = coordString.split('|')
    for item in items:
        if '=' in item:
            keywords[item.split('=')[0]] = item.split('=')[1]
        elif ':' in item:
            keywords[item.split(':')[0]] = item.split(':')[1]
        else:
            rest.append(item)
#     return keywords, '|'.join(rest)
    return keywords
print(getKeywords('Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing'))
print(getKeywords('Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'))
print(getKeywords('Coord|36|42|N|3|13|E|type:city'))

{'type': 'event', 'name': 'Apollo 8 landing'}
{'region': 'US-AK_type', 'display': 'title', 'notes': '<ref>{{Cite gnis'}
{'type': 'city'}


In [144]:
from collections import defaultdict
keywords = defaultdict(int)
for coordString in coordStrings.values():
    for kw in getKeywords(coordString).keys():
        keywords[kw.strip()] += 1
keywords

defaultdict(int,
            {'type': 56807,
             'display': 118395,
             'region': 29012,
             'notes': 970,
             'name': 5848,
             'globe': 228,
             'scale': 2087,
             'source': 4384,
             'format': 12008,
             'dim': 524,
             '3': 3,
             '4': 4,
             '6': 1,
             '7': 1,
             'label': 1,
             'url': 68,
             'title': 76,
             'accessdate': 67,
             'DK_type': 1,
             "The ''globe''&nbsp;[[File": 25,
             'upright': 25,
             '': 1,
             'date': 29,
             'work': 42,
             'publisher': 56,
             'USGS': 1,
             'Register of Historic Parks and Gardens]].<ref name': 1,
             'num': 1,
             'desc': 1,
             'access-date': 25,
             'mode': 1,
             'deadurl': 2,
             'archiveurl': 1,
             'archivedate': 1,
             'df': 1,
  

In [145]:
{kw for kw,count in keywords.items() if count > 10}

{"The ''globe''&nbsp;[[File",
 'access-date',
 'accessdate',
 'date',
 'dim',
 'display',
 'format',
 'globe',
 'id',
 'name',
 'notes',
 'publisher',
 'region',
 'scale',
 'source',
 'title',
 'type',
 'upright',
 'url',
 'work'}

In [146]:
entriesList = [
 'accessdate',
 'date',
 'dim',
 'display',
 'elevation',
 'format',
 'globe',
 'id',
 'name',
 'nosave',
 'notes',
 'publisher',
 'reason',
 'region',
 'scale',
 'source',
 'title',
 'type',
 'upright',
 'url',
 'work']
for e in entriesList:
    print(e, 'TEXT,')

accessdate TEXT,
date TEXT,
dim TEXT,
display TEXT,
elevation TEXT,
format TEXT,
globe TEXT,
id TEXT,
name TEXT,
nosave TEXT,
notes TEXT,
publisher TEXT,
reason TEXT,
region TEXT,
scale TEXT,
source TEXT,
title TEXT,
type TEXT,
upright TEXT,
url TEXT,
work TEXT,


In [150]:
from numpy.random import shuffle
def findKeywordExample(kw, count=1, debug=False):
    kws = []
    coordStringsList = list(coordStrings.values())
    shuffle(coordStringsList)
    for cs in coordStringsList:
        if f'|{kw}=' in cs:
            kws.append(cs.split(f'|{kw}=')[1].split('|')[0])
            if debug:
                print(cs)
        elif f'|{kw}:' in cs:
            kws.append(cs.split(f'|{kw}:')[1].split('|')[0])
            if debug:
                print(cs)
        if len(kws) >= count:
            break
    return kws

In [151]:
findKeywordExample('name', 10, True)

Coord|32|57|55|N|044|16|44|E|name=Al-Iskandariyah New AB
Coord|36.74|N|48.87|E|globe:moon|name=Berzelius A
Coord|-26.0055439|28.2102084|type:landmark|name=Mehlareng Stadium|display=inline,title
Coord|58|55|51|N|2|59|00|W|name=HMS Royal Oak (08)
Coord|43|0|N|0|0|E|type:landmark|name=Prime Meridian
Coord|53.8533|-2.2113|name=Pendle Heritage Centre
Coord|28|22|N|082|33|E|type:landmark_scale:250000|name=Rolpa District, Nepal|display=title
Coord|38.971364|N|77.043774|W|name=Pinehurst Bridge|type:landmark_region:US-DC
Coord|54.711|N|111.966|W|scale:60000_region:CA|display=title|name=Bone Town
Coord|display=title|name=Wadena, Saskatchewan|51|56|45|N|103|48|05|W|region:CA_type:city_source:GNS-enwiki


['Al-Iskandariyah New AB',
 'Berzelius A',
 'Mehlareng Stadium',
 'HMS Royal Oak (08)',
 'Prime Meridian',
 'Pendle Heritage Centre',
 'Rolpa District, Nepal',
 'Pinehurst Bridge',
 'Bone Town',
 'Wadena, Saskatchewan']

In [228]:
c.execute('''CREATE TABLE coords
    (coords TEXT,
    page_num INTEGER PRIMARY KEY,
    accessdate TEXT DEFAULT '',
    date TEXT DEFAULT '',
    dim TEXT DEFAULT '',
    display TEXT DEFAULT '',
    elevation TEXT DEFAULT '',
    format TEXT DEFAULT '',
    globe TEXT DEFAULT '',
    id TEXT DEFAULT '',
    name TEXT DEFAULT '',
    nosave TEXT DEFAULT '',
    notes TEXT DEFAULT '',
    publisher TEXT DEFAULT '',
    reason TEXT DEFAULT '',
    region TEXT DEFAULT '',
    scale TEXT DEFAULT '',
    source TEXT DEFAULT '',
    title TEXT DEFAULT '',
    type TEXT DEFAULT '',
    upright TEXT DEFAULT '',
    url TEXT DEFAULT '',
    work TEXT DEFAULT '')
''')

In [227]:
c.execute('''DROP TABLE coords''')

In [225]:
def insertKeywordDict(page_num, coords, kws, debug=False):
    allowed_keys = [kw for kw in kws.keys() if kw in entriesList]
    allowed_vals = [kws[kw] for kw in allowed_keys]
    
    keys = ['coords', 'page_num'] + allowed_keys
    vals = [coords, page_num] + allowed_vals
    
    qstring = f'INSERT INTO coords (\
        {",".join(keys)}) VALUES (\
        {",".join(["?" for item in vals])})'
    
    if debug:
        print(qstring, vals)
    c.execute(qstring, vals)
# insertKeywordDict({'region': 'US-AK_type', 'display': 'title', 'notes': '<ref>{{Cite gnis'})

In [229]:
count = 0
for page_num,coordString in coordStrings.items():
    keywords = getKeywords(coordString)
    if page_num > 999999999:
        insertKeywordDict(page_num, coordString, keywords, True)
    else:
        insertKeywordDict(page_num, coordString, keywords)
    if count > 100:
        english.db.commit()
        print(round(100*count/len(coordStrings), 2), '%', end='\r')
    count += 1

In [178]:
# c.execute('''SELECT * FROM coords WHERE page_num > 19000000''').fetchall()

In [236]:
c.execute('''SELECT page_num,type FROM coords WHERE page_num > 19060000''').fetchall()

[(19063412, 'landmark'),
 (19063772, 'landmark_region'),
 (19063981, 'city_region'),
 (19064246, 'landmark_dim'),
 (19064477, 'landmark_region'),
 (19064496, ''),
 (19064535, ''),
 (19064542, ''),
 (19064563, ''),
 (19064572, ''),
 (19064626, 'city_region'),
 (19064881, 'airport_region'),
 (19064893, 'landmark'),
 (19064974, 'landmark'),
 (19065216, ''),
 (19065236, ''),
 (19065252, ''),
 (19065273, ''),
 (19065287, ''),
 (19065308, ''),
 (19065309, 'landmark'),
 (19065314, ''),
 (19065324, ''),
 (19065374, ''),
 (19065394, 'waterbody_region'),
 (19065421, ''),
 (19065427, ''),
 (19065439, ''),
 (19065452, ''),
 (19065511, ''),
 (19065516, ''),
 (19065564, ''),
 (19065577, ''),
 (19065684, ''),
 (19065802, ''),
 (19065808, ''),
 (19065822, ''),
 (19065862, ''),
 (19065876, ''),
 (19065888, ''),
 (19065922, ''),
 (19065925, ''),
 (19065958, ''),
 (19066036, ''),
 (19066058, ''),
 (19066076, ''),
 (19066637, 'landmark'),
 (19066932, 'landmark_region'),
 (19066942, 'landmark_region'),
 (1